# Team Members
# Tarik Koric netid:koric1
# Pranav Velamakanni netid: pranavv2

## Pranav: 
- derivationn
- e-step


## Tarik:
- derivation
- m-step

In [339]:
set.seed(2296)

In [340]:
##################################################################
# Public functions
##################################################################
#' Multivariate Normal Density 
#' 
#' Calculates mutlivariate normal densities. This is different to the dmvnorm function 
#' in the mvtnorm package in that it takes a matrix for both x and mean. It then calculates 
#' a vector of densities according to dmvnorm(x[i,],mean[i,],sigma,log = FALSE).
#' To aid computation the mahalanobis distances are calculated in parallel using mclapply.
#' @importFrom parallel mclapply
#' @param x A matrix of values
#' @param mean A matrix of means
#' @param sigma A covariance matrix
#' @param log Boolean for whether we want log densities or not
#' @details My own implementation of the multivariate normal density function
#' for increased efficiency for application in this package
#' because there are so many repeated calls to densitymvnorm using a 
#' given sigma matrix it made sense to have one that could take a matrix 
#' of means as well as a matrix of x's and treat them as 
#' paired, returning the density of x[1,] given mean[1,], x[2,] 
#' given mean[2,] also stops repeated inversions of the matrix sigma, and
#' calculates the densities in parallel
#' @return A vector of densities
#' @export
densityMvNorm = function (x, mean, sigma, log = FALSE) 
{
  ## takes a matrix of means rather than a single vector
  if (missing(sigma)) sigma = diag(ncol(x))
  
  if (NCOL(x) != NCOL(sigma)) {
    stop("x and sigma have non-conforming size")
  }
  if (!isSymmetric(sigma, tol = sqrt(.Machine$double.eps), 
                   check.attributes = FALSE)) {
    stop("sigma must be a symmetric matrix")
  }
  if (NCOL(mean) != NROW(sigma)) {
    stop("mean and sigma have non-conforming size")
  }
  ## invert matrix before hand so only do this once
  prec = solve(sigma)
  means = lapply(1:dim(mean)[1],function(i){mean[i,]})
  distval = do.call(rbind, 
                     mclapply(means, 
                              mahalanobis,x = x, cov = prec,inverted = TRUE))
  logdet = sum(log(eigen(sigma, symmetric = TRUE, only.values = TRUE)$values))
  logretval = -(ncol(x) * log(2 * pi) + logdet + distval)/2
  
  if (log) 
    return(logretval)
  else 
    return(exp(logretval))
}

In [341]:
Estep <- function(data, G, para){
  # Your Code
  # Return the n-by-G probability matrix
    n = dim(data)[1]
    prob = matrix(0,n,G)
    x = matrix(0,n,G)
    
    #A = t(densityMvNorm(faithful,t(para$mean),para$Sigma))
    x[,1] = t(densityMvNorm(data, t(para$mean), para$Sigma))[,1] * para$prob[1]
    x[,2] = t(densityMvNorm(data, t(para$mean), para$Sigma))[,2] * para$prob[2]
    
    prob[,1] = x[,1]/(x[,1]+x[,2])
    prob[,2] = x[,2]/(x[,1]+x[,2])
    #for (x in 1:n){
        
       # prob[x,1] = A[x,1]/(A[x,1] + A[x,2])
        #prob[x,2] = A[x,2]/(A[x,1] + A[x,2])
    #}
    
    #print(1)
    return (prob)
  }

Mstep <- function(data, G, para, post.prob){ 
  # Your Code
  # Return the updated parameters
    
    n = dim(data)[1]
    
    para$prob[1] = mean(post.prob[,1])
    para$prob[2] = mean(post.prob[,2])
    
    para$mean[1,1] = sum(post.prob[,1]*data[,1])/sum(post.prob[,1])
    para$mean[1,2] = sum(post.prob[,2]*data[,1])/sum(post.prob[,2])
    para$mean[2,1] = sum(post.prob[,1]*data[,2])/sum(post.prob[,1])
    para$mean[2,2] = sum(post.prob[,2]*data[,2])/sum(post.prob[,2])
    
    x = matrix(0,2,2)
    y = matrix(0,2,2)
    
    for (i in 1:n){
        
        a = t((data[i,]-para$mean[,1])*post.prob[i,1])
        b = (data[i,]-para$mean[,1])
        
        x[1,1] = (a[1,1] * b[1,1]) + x[1,1]
        x[1,2] = (a[1,1] * b[1,2]) + x[1,2]
        x[2,1] = (a[2,1] * b[1,1]) + x[2,1]
        x[2,2] = (a[2,1] * b[1,2]) + x[2,2]
        
        
        a = t((data[i,]-para$mean[,2])*post.prob[i,2])
        b = (data[i,]-para$mean[,2])
        
        y[1,1] = (a[1,1] * b[1,1]) + y[1,1]
        y[1,2] = (a[1,1] * b[1,2]) + y[1,2]
        y[2,1] = (a[2,1] * b[1,1]) + y[2,1]
        y[2,2] = (a[2,1] * b[1,2]) + y[2,2]
        
    }
    
    para$Sigma = (x+y)/272
    
    return(para)
    
    
    
  }

myEM <- function(data, itmax, G, para){
  # itmax: num of iterations
  # G:     num of components
  # para:  list of parameters (prob, mean, Sigma)
  for(t in 1:itmax){
    post.prob <- Estep(data, G, para)
    para <- Mstep(data, G, para, post.prob)
  }
  return(para)
}

In [342]:
Z <- matrix(0, n, 2) 
Z[sample(1:n, 120), 1] <- 1 
Z[, 2] <- 1 - Z[, 1]
ini0 <- mstep(modelName="EEE", faithful , Z)$parameters

In [343]:
para0 <- list(prob = ini0$pro, 
              mean = ini0$mean, 
              Sigma = ini0$variance$Sigma)
para0

eruptions,3.472325,3.499987
waiting,70.733333,71.026316
,eruptions,waiting
eruptions,1.29775,13.92442
waiting,13.92442,184.12265


In [344]:
myEM(data=faithful, itmax=10, G=2, para=para0)

$prob
[1] 0.4411753 0.5588247

$mean
               [,1]      [,2]
eruptions  3.472223  3.500067
waiting   70.732092 71.027296

$Sigma
          [,1]      [,2]
[1,]  1.297748  13.92439
[2,] 13.924392 184.12233

In [345]:
Rout <- em(modelName = "EEE", data = faithful,
           control = emControl(eps=0, tol=0, itmax = 10), 
           parameters = ini0)$parameters
list(Rout$pro, Rout$mean, Rout$variance$Sigma)

eruptions,3.472223,3.500067
waiting,70.732092,71.027296
,eruptions,waiting
eruptions,1.297748,13.92439
waiting,13.924392,184.12233
